# PDF Parsing Isolation - ICC Judgment Processing

This notebook isolates the PDF parsing logic from the main RAG pipeline, providing a dedicated, reusable PDF processing system for ICC judgments.

## 🎯 Purpose
- **Isolated PDF parsing** using PyMuPDF (winner from local testing)
- **Modular design** for easy integration with other notebooks
- **Production-ready** with comprehensive error handling
- **Optimized for legal documents** with ICC-specific patterns

## 📊 Library Comparison Results
Based on local testing with the actual ICC judgment:
- **PyMuPDF**: 28.8x faster, 4,475,751 chars extracted
- **pdfplumber**: 141.7s extraction time, 4,407,110 chars extracted
- **Winner**: PyMuPDF for speed and text quality

## 🔄 Workflow Integration
This notebook is designed to be the first step in the pipeline:
1. **Parse PDF** → This notebook
2. **Build Chunks** → 03_ICC_Judgment_Chunking.ipynb
3. **Summarize Chunks** → 04_ICC_Judgment_Chunking_summary.ipynb
4. **Optimize Vector Search** → 01_Optimized_Vector_Search.ipynb
5. **Build and Deploy RAG** → 02_Production_RAG_Deployment.ipynb


In [ ]:
# Install required packages
%pip install PyMuPDF pandas pyarrow


In [ ]:
# Load configuration and imports
import sys
sys.path.append('/Workspace/Users/christophe629@gmail.com/icc_rag_backend/databricks-deployment/config')

# Import unified configuration
from databricks_config import *

# Core imports
import fitz  # PyMuPDF
import json
import re
import pandas as pd
from typing import Dict, List, Tuple, Optional, Set
from dataclasses import dataclass, asdict
from collections import defaultdict
from pathlib import Path
import time
import logging
import builtins

# Spark imports
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

print("✅ Configuration and imports loaded")
print_config_summary()


## Data Models for PDF Parsing

Structured data models to maintain consistency and type safety across the parsing pipeline:


In [ ]:
@dataclass
class ParsedPage:
    """Represents a single parsed page from the PDF."""
    page_number: int
    text: str
    text_length: int
    word_count: int
    section_type: str
    has_images: bool
    image_count: int
    extraction_quality: float  # 0.0-1.0
    processing_time: float  # seconds

@dataclass
class LegalParagraph:
    """Represents a legal paragraph with numbering."""
    paragraph_id: str
    paragraph_number: Optional[str]  # [123] format
    content: str
    page_number: int
    section_type: str
    word_count: int
    char_count: int
    is_numbered: bool
    extraction_confidence: float

@dataclass
class DocumentMetadata:
    """Document-level metadata extracted from PDF."""
    case_name: str
    case_number: str
    chamber: str
    date: str
    total_pages: int
    total_text_length: int
    total_word_count: int
    extraction_method: str
    processing_time: float
    quality_score: float

@dataclass
class ParsedDocument:
    """Complete parsed document structure."""
    metadata: DocumentMetadata
    pages: List[ParsedPage]
    paragraphs: List[LegalParagraph]
    sections: Dict[str, List[int]]  # section_type -> page_numbers
    extraction_errors: List[str]

print("✅ Data models defined")


## ICC PDF Parser Class

Production-ready PDF parser optimized for ICC judgments using PyMuPDF:


In [ ]:
class ICCPDFParser:
    """Production PDF parser optimized for ICC judgments using PyMuPDF."""
    
    def __init__(self, config: Dict = None):
        """Initialize parser with configuration."""
        self.config = config or {}
        
        # ICC-specific patterns for section identification
        self.section_patterns = [
            ('OVERVIEW', r'I\.\s+OVERVIEW'),
            ('FINDINGS_OF_FACT', r'II\.\s+FINDINGS\s+OF\s+FACT'),
            ('EVIDENTIARY_CONSIDERATIONS', r'III\.\s+EVIDENTIARY\s+CONSIDERATIONS'),
            ('APPLICABLE_LAW', r'IV\.\s+APPLICABLE\s+LAW'),
            ('LEGAL_CHARACTERISATION', r'V\.\s+LEGAL\s+CHARACTERISATION'),
            ('SENTENCE', r'VI\.\s+SENTENCE'),
            ('VERDICT', r'VII\.\s+VERDICT'),
        ]
        
        # Legal paragraph patterns
        self.paragraph_patterns = [
            r'\[(\d+)\]\s*([^[]*?)(?=\[|\Z)',  # [123] format
            r'^\s*(\d{1,3})\.\s+([^0-9].*?)(?=\n\s*\d+\.|\Z)',  # 123. format
        ]
        
        # Document metadata patterns
        self.metadata_patterns = {
            'case_name': r'Prosecutor\s+v\.\s+([^,]+)',
            'case_number': r'(ICC-\d+/\d+)',
            'chamber': r'(Trial\s+Chamber\s+[IVX]+|Appeals\s+Chamber)',
            'date': r'(\d{1,2}\s+\w+\s+\d{4})',
        }
        
        # Quality assessment patterns
        self.quality_indicators = {
            'legal_paragraphs': r'\[\d+\]',
            'case_citations': r'ICC-\d+/\d+',
            'section_headers': r'^[IVX]+\.\s+[A-Z\s]+$',
            'legal_terms': r'(war\s+crime|crimes\s+against\s+humanity|genocide|persecution)',
        }
    
    def parse_pdf(self, pdf_path: str) -> ParsedDocument:
        """Parse PDF and return structured document."""
        print(f"🔍 Parsing PDF: {pdf_path}")
        start_time = time.time()
        
        try:
            # Open PDF with PyMuPDF
            doc = fitz.open(pdf_path)
            
            # Extract document metadata
            metadata = self._extract_document_metadata(doc, pdf_path)
            
            # Parse all pages
            pages = []
            paragraphs = []
            sections = defaultdict(list)
            errors = []
            
            for page_num in range(doc.page_count):
                try:
                    page_start = time.time()
                    page = doc[page_num]
                    
                    # Extract page text
                    text = page.get_text("text")
                    
                    # Identify section type
                    section_type = self._identify_section_type(text)
                    sections[section_type].append(page_num + 1)
                    
                    # Extract images info
                    images = page.get_images()
                    has_images = len(images) > 0
                    
                    # Calculate extraction quality
                    quality = self._calculate_page_quality(text)
                    
                    # Create page object
                    parsed_page = ParsedPage(
                        page_number=page_num + 1,
                        text=text,
                        text_length=len(text),
                        word_count=len(text.split()),
                        section_type=section_type,
                        has_images=has_images,
                        image_count=len(images),
                        extraction_quality=quality,
                        processing_time=time.time() - page_start
                    )
                    pages.append(parsed_page)
                    
                    # Extract legal paragraphs
                    page_paragraphs = self._extract_legal_paragraphs(text, page_num + 1, section_type)
                    paragraphs.extend(page_paragraphs)
                    
                except Exception as e:
                    error_msg = f"Page {page_num + 1}: {str(e)}"
                    errors.append(error_msg)
                    print(f"⚠️  {error_msg}")
            
            # Update metadata with actual counts
            metadata.total_pages = len(pages)
            metadata.total_text_length = sum(p.text_length for p in pages)
            metadata.total_word_count = sum(p.word_count for p in pages)
            metadata.processing_time = time.time() - start_time
            metadata.quality_score = self._calculate_document_quality(pages, paragraphs)
            
            # Create parsed document
            parsed_doc = ParsedDocument(
                metadata=metadata,
                pages=pages,
                paragraphs=paragraphs,
                sections=dict(sections),
                extraction_errors=errors
            )
            
            doc.close()
            
            print(f"✅ PDF parsing complete: {len(pages)} pages, {len(paragraphs)} paragraphs")
            print(f"⏱️  Processing time: {metadata.processing_time:.2f}s")
            print(f"🎯 Quality score: {metadata.quality_score:.2f}/100")
            
            return parsed_doc
            
        except Exception as e:
            print(f"❌ PDF parsing failed: {str(e)}")
            raise
    
    def _extract_document_metadata(self, doc: fitz.Document, pdf_path: str) -> DocumentMetadata:
        """Extract document-level metadata."""
        # Try to extract from first few pages
        metadata_text = ""
        for page_num in range(builtins.min(3, doc.page_count)):
            page = doc[page_num]
            metadata_text += page.get_text("text") + "\n"
        
        # Extract using patterns
        case_name = "Unknown Case"
        case_number = "Unknown"
        chamber = "Unknown Chamber"
        date = "Unknown Date"
        
        for pattern_name, pattern in self.metadata_patterns.items():
            match = re.search(pattern, metadata_text, re.IGNORECASE)
            if match:
                if pattern_name == 'case_name':
                    case_name = match.group(1).strip()
                elif pattern_name == 'case_number':
                    case_number = match.group(1).strip()
                elif pattern_name == 'chamber':
                    chamber = match.group(1).strip()
                elif pattern_name == 'date':
                    date = match.group(1).strip()
        
        return DocumentMetadata(
            case_name=case_name,
            case_number=case_number,
            chamber=chamber,
            date=date,
            total_pages=0,  # Will be updated later
            total_text_length=0,  # Will be updated later
            total_word_count=0,  # Will be updated later
            extraction_method="PyMuPDF",
            processing_time=0.0,  # Will be updated later
            quality_score=0.0  # Will be updated later
        )
    
    def _identify_section_type(self, text: str) -> str:
        """Identify the section type based on text content."""
        text_upper = text.upper()
        
        for section_name, pattern in self.section_patterns:
            if re.search(pattern, text_upper):
                return section_name
        
        # Check for common headers
        if any(header in text_upper for header in ['TRIAL CHAMBER', 'APPEALS CHAMBER']):
            return 'HEADER'
        
        return 'UNKNOWN'
    
    def _extract_legal_paragraphs(self, text: str, page_number: int, section_type: str) -> List[LegalParagraph]:
        """Extract legal paragraphs from page text."""
        paragraphs = []
        
        for pattern in self.paragraph_patterns:
            for match in re.finditer(pattern, text, re.DOTALL):
                para_num = match.group(1)
                para_content = match.group(2).strip()
                
                if len(para_content) > 20:  # Skip very short paragraphs
                    # Clean content
                    clean_content = re.sub(r'\s+', ' ', para_content).strip()
                    
                    # Calculate confidence
                    confidence = self._calculate_paragraph_confidence(clean_content)
                    
                    paragraph = LegalParagraph(
                        paragraph_id=f"para_{section_type}_{page_number}_{para_num}",
                        paragraph_number=para_num,
                        content=clean_content,
                        page_number=page_number,
                        section_type=section_type,
                        word_count=len(clean_content.split()),
                        char_count=len(clean_content),
                        is_numbered=True,
                        extraction_confidence=confidence
                    )
                    paragraphs.append(paragraph)
        
        return paragraphs
    
    def _calculate_page_quality(self, text: str) -> float:
        """Calculate extraction quality for a page."""
        score = 0.0
        
        # Check for legal document indicators
        for indicator, pattern in self.quality_indicators.items():
            matches = len(re.findall(pattern, text, re.IGNORECASE))
            if matches > 0:
                score += builtins.min(matches * 0.1, 0.3)  # Cap at 0.3 per indicator
        
        # Length bonus
        if len(text) > 1000:
            score += 0.2
        elif len(text) > 500:
            score += 0.1
        
        return builtins.min(score, 1.0)
    
    def _calculate_paragraph_confidence(self, content: str) -> float:
        """Calculate confidence score for a paragraph."""
        score = 0.0
        
        # Legal content indicators
        legal_terms = ['court', 'chamber', 'prosecutor', 'defendant', 'evidence', 'witness', 'crime']
        legal_count = sum(1 for term in legal_terms if term.lower() in content.lower())
        score += builtins.min(legal_count * 0.1, 0.4)
        
        # Length penalty for very short content
        if len(content) < 50:
            score -= 0.2
        
        # Citation patterns
        if re.search(r'ICC-\d+/\d+', content):
            score += 0.2
        
        return builtins.max(0.0, builtins.min(score, 1.0))
    
    def _calculate_document_quality(self, pages: List[ParsedPage], paragraphs: List[LegalParagraph]) -> float:
        """Calculate overall document quality score."""
        if not pages:
            return 0.0
        
        # Average page quality
        avg_page_quality = sum(p.extraction_quality for p in pages) / len(pages)
        
        # Paragraph quality bonus
        if paragraphs:
            avg_para_confidence = sum(p.extraction_confidence for p in paragraphs) / len(paragraphs)
            quality_score = (avg_page_quality * 0.7) + (avg_para_confidence * 0.3)
        else:
            quality_score = avg_page_quality
        
        return builtins.min(quality_score * 100, 100.0)

print("✅ ICC PDF Parser class defined")


## Data Export and Integration Functions

Functions to export parsed data and integrate with the chunking pipeline:


In [ ]:
def export_parsed_document(parsed_doc: ParsedDocument, output_format: str = "json") -> str:
    """Export parsed document in various formats."""
    
    if output_format.lower() == "json":
        # Convert to JSON-serializable format
        export_data = {
            "metadata": asdict(parsed_doc.metadata),
            "pages": [asdict(page) for page in parsed_doc.pages],
            "paragraphs": [asdict(para) for para in parsed_doc.paragraphs],
            "sections": parsed_doc.sections,
            "extraction_errors": parsed_doc.extraction_errors
        }
        
        output_path = f"/tmp/parsed_document_{int(time.time())}.json"
        with open(output_path, 'w', encoding='utf-8') as f:
            json.dump(export_data, f, indent=2, ensure_ascii=False)
        
        return output_path
    
    elif output_format.lower() == "parquet":
        # Create Spark DataFrame and save as Parquet
        spark = SparkSession.getActiveSession()
        if spark is None:
            spark = SparkSession.builder.appName("PDF_Parsing").getOrCreate()
        
        # Convert pages to DataFrame
        pages_data = []
        for page in parsed_doc.pages:
            pages_data.append({
                "page_number": page.page_number,
                "text": page.text,
                "text_length": page.text_length,
                "word_count": page.word_count,
                "section_type": page.section_type,
                "has_images": page.has_images,
                "image_count": page.image_count,
                "extraction_quality": page.extraction_quality,
                "processing_time": page.processing_time
            })
        
        pages_df = spark.createDataFrame(pages_data)
        
        output_path = f"/tmp/parsed_pages_{int(time.time())}.parquet"
        pages_df.write.mode("overwrite").parquet(output_path)
        
        return output_path
    
    else:
        raise ValueError(f"Unsupported output format: {output_format}")

def create_spark_dataframe(parsed_doc: ParsedDocument) -> 'pyspark.sql.DataFrame':
    """Create Spark DataFrame from parsed document for integration with chunking pipeline."""
    spark = SparkSession.getActiveSession()
    if spark is None:
        spark = SparkSession.builder.appName("PDF_Parsing").getOrCreate()
    
    # Prepare data for chunking pipeline
    chunking_data = []
    
    for page in parsed_doc.pages:
        # Create base record for each page
        base_record = {
            "page_number": page.page_number,
            "text": page.text,
            "section_type": page.section_type,
            "text_length": page.text_length,
            "word_count": page.word_count,
            "extraction_quality": page.extraction_quality,
            "case_name": parsed_doc.metadata.case_name,
            "case_number": parsed_doc.metadata.case_number,
            "chamber": parsed_doc.metadata.chamber,
            "date": parsed_doc.metadata.date,
            "extraction_method": parsed_doc.metadata.extraction_method
        }
        
        # Add paragraph-specific data if available
        page_paragraphs = [p for p in parsed_doc.paragraphs if p.page_number == page.page_number]
        
        if page_paragraphs:
            # Create record for each paragraph
            for para in page_paragraphs:
                record = base_record.copy()
                record.update({
                    "paragraph_id": para.paragraph_id,
                    "paragraph_number": para.paragraph_number,
                    "paragraph_content": para.content,
                    "paragraph_word_count": para.word_count,
                    "paragraph_char_count": para.char_count,
                    "is_numbered": para.is_numbered,
                    "extraction_confidence": para.extraction_confidence
                })
                chunking_data.append(record)
        else:
            # No paragraphs found, use page-level data
            record = base_record.copy()
            record.update({
                "paragraph_id": f"page_{page.page_number}",
                "paragraph_number": None,
                "paragraph_content": page.text,
                "paragraph_word_count": page.word_count,
                "paragraph_char_count": page.text_length,
                "is_numbered": False,
                "extraction_confidence": page.extraction_quality
            })
            chunking_data.append(record)
    
    # Create DataFrame
    df = spark.createDataFrame(chunking_data)
    return df

def save_to_delta_table(parsed_doc: ParsedDocument, table_name: str = None) -> str:
    """Save parsed document to Delta table for downstream processing."""
    spark = SparkSession.getActiveSession()
    if spark is None:
        spark = SparkSession.builder.appName("PDF_Parsing").getOrCreate()
    
    # Create DataFrame
    df = create_spark_dataframe(parsed_doc)
    
    # Use configured table name or default
    if table_name is None:
        table_name = get_databricks_path("parsed_document_table")
    
    # Save to Delta table
    df.write.format("delta").mode("overwrite").saveAsTable(table_name)
    
    print(f"✅ Parsed document saved to Delta table: {table_name}")
    print(f"📊 Records: {df.count()}")
    
    return table_name

print("✅ Export and integration functions defined")


## Usage Examples and Testing

Examples of how to use the PDF parsing system:


In [ ]:
# Example 1: Basic PDF Parsing
def parse_icc_judgment(pdf_path: str, save_to_delta: bool = True) -> ParsedDocument:
    """Parse ICC judgment PDF and optionally save to Delta table."""
    
    # Initialize parser
    parser = ICCPDFParser()
    
    # Parse the PDF
    parsed_doc = parser.parse_pdf(pdf_path)
    
    # Display results
    print(f"\n📊 PARSING RESULTS:")
    print(f"   Case: {parsed_doc.metadata.case_name}")
    print(f"   Number: {parsed_doc.metadata.case_number}")
    print(f"   Chamber: {parsed_doc.metadata.chamber}")
    print(f"   Date: {parsed_doc.metadata.date}")
    print(f"   Pages: {parsed_doc.metadata.total_pages}")
    print(f"   Text length: {parsed_doc.metadata.total_text_length:,} chars")
    print(f"   Word count: {parsed_doc.metadata.total_word_count:,} words")
    print(f"   Quality score: {parsed_doc.metadata.quality_score:.1f}/100")
    print(f"   Processing time: {parsed_doc.metadata.processing_time:.2f}s")
    
    # Section breakdown
    print(f"\n📋 SECTIONS FOUND:")
    for section, pages in parsed_doc.sections.items():
        print(f"   {section}: {len(pages)} pages")
    
    # Paragraph statistics
    if parsed_doc.paragraphs:
        print(f"\n📝 PARAGRAPHS EXTRACTED:")
        print(f"   Total paragraphs: {len(parsed_doc.paragraphs)}")
        numbered_paras = [p for p in parsed_doc.paragraphs if p.is_numbered]
        print(f"   Numbered paragraphs: {len(numbered_paras)}")
        avg_confidence = sum(p.extraction_confidence for p in parsed_doc.paragraphs) / len(parsed_doc.paragraphs)
        print(f"   Average confidence: {avg_confidence:.3f}")
    
    # Errors
    if parsed_doc.extraction_errors:
        print(f"\n⚠️  EXTRACTION ERRORS: {len(parsed_doc.extraction_errors)}")
        for error in parsed_doc.extraction_errors[:5]:  # Show first 5 errors
            print(f"   {error}")
    
    # Save to Delta table if requested
    if save_to_delta:
        table_name = save_to_delta_table(parsed_doc)
        print(f"\n💾 Saved to Delta table: {table_name}")
    
    return parsed_doc

print("✅ Basic parsing function defined")


In [ ]:
# Example 2: Parse the ICC Judgment
pdf_path = PDF_SOURCE_PATH

print(f"🔍 Parsing ICC Judgment: {pdf_path}")
print("=" * 50)

# Parse the document
parsed_document = parse_icc_judgment(pdf_path, save_to_delta=True)

print("\n✅ PDF parsing complete!")
print("📋 Ready for next step: Chunking pipeline")


## Integration with Chunking Pipeline

Functions to integrate parsed data with the chunking system:


In [ ]:
def prepare_for_chunking(parsed_doc: ParsedDocument) -> 'pyspark.sql.DataFrame':
    """Prepare parsed document data for the chunking pipeline."""
    
    # Create Spark DataFrame
    df = create_spark_dataframe(parsed_doc)
    
    # Add additional fields needed for chunking
    df = df.withColumn("document_id", lit(f"icc_{parsed_doc.metadata.case_number.replace('/', '_')}"))
    df = df.withColumn("extraction_timestamp", current_timestamp())
    df = df.withColumn("pipeline_stage", lit("parsed"))
    
    # Add quality indicators
    df = df.withColumn("high_quality", 
                      when(col("extraction_quality") > 0.7, True).otherwise(False))
    df = df.withColumn("has_legal_content", 
                      when(col("paragraph_content").contains("court") | 
                           col("paragraph_content").contains("chamber") |
                           col("paragraph_content").contains("prosecutor"), True).otherwise(False))
    
    print(f"✅ Prepared {df.count()} records for chunking pipeline")
    print(f"📊 High quality records: {df.filter(col('high_quality')).count()}")
    print(f"⚖️  Legal content records: {df.filter(col('has_legal_content')).count()}")
    
    return df

def export_for_chunking(parsed_doc: ParsedDocument, output_table: str = None) -> str:
    """Export parsed data in format ready for chunking pipeline."""
    
    # Prepare data
    df = prepare_for_chunking(parsed_doc)
    
    # Use configured table name or default
    if output_table is None:
        output_table = get_databricks_path("parsed_for_chunking")
    
    # Save to Delta table
    df.write.format("delta").mode("overwrite").saveAsTable(output_table)
    
    print(f"✅ Data exported for chunking: {output_table}")
    
    return output_table

# Example: Prepare data for chunking
if 'parsed_document' in locals():
    print("🔄 Preparing data for chunking pipeline...")
    chunking_table = export_for_chunking(parsed_document)
    print(f"📋 Next step: Use table '{chunking_table}' in chunking notebook")
else:
    print("ℹ️  Run the parsing example above first to prepare data for chunking")


## Summary and Next Steps

This notebook provides a complete, isolated PDF parsing system for ICC judgments:

### ✅ What's Included
- **PyMuPDF-based parser** (28.8x faster than pdfplumber)
- **ICC-specific patterns** for legal document structure
- **Quality assessment** with confidence scoring
- **Multiple export formats** (JSON, Parquet, Delta)
- **Spark integration** for downstream processing
- **Error handling** and comprehensive logging

### 🔄 Workflow Integration
1. **Parse PDF** → This notebook (00_PDF_Parsing_Isolation.ipynb)
2. **Build Chunks** → 03_ICC_Judgment_Chunking.ipynb
3. **Summarize Chunks** → 04_ICC_Judgment_Chunking_summary.ipynb
4. **Optimize Vector Search** → 01_Optimized_Vector_Search.ipynb
5. **Build and Deploy RAG** → 02_Production_RAG_Deployment.ipynb

### 📊 Performance Metrics
- **Speed**: ~5 seconds for 1,616 pages (vs 141s with pdfplumber)
- **Text extraction**: 4,475,751 characters
- **Quality score**: Automated assessment of extraction quality
- **Error handling**: Comprehensive error tracking and reporting

### 🚀 Ready for Production
The parsed data is automatically saved to Delta tables and ready for the next stage of the RAG pipeline.
